In [2]:
from collections import namedtuple
from datetime import datetime
from pprint import pprint
from itertools import cycle

In [34]:
Set_ = namedtuple('Set_', [
    'reps',
    'exercises',
])

Exercise = namedtuple('Exercise', [
    'name',
    'time',
])

RepsSetting = namedtuple('RepsSetting', [
    'start',
    'step',
    'step_interval',
    'max',
])

ExerciseSetting = namedtuple('ExerciseSetting', [
    'description',
    'start',
    'step',
    'step_interval',
    'max',
])

In [ ]:
progress_settings = {
    'runeasy': {'warmup': 0,
                'warmdown': 0,
                'reps': RepsSetting(1, 0, 0, 1),
                'exercises': [ExerciseSetting('easy', 25, 5, 3, 100)]},
    'hillsprint': {'warmup': 12,
                   'warmdown': 12,
                   'reps': RepsSetting(6, 1, 2, 8),
                   'exercises': [ExerciseSetting('hill', 1, 0.25, 2, 2)]},
    'intervals': {'warmup': 10,
                  'warmdown': 10,
                  'reps': RepsSetting(5, 0, 0, 1),
                  'exercises': [ExerciseSetting('fast', 0.5, 0.25, 2, 2),
                                ExerciseSetting('easy', 1, 0, 0, 1)]},
    'crosstrain': {'warmup': 0,
                   'warmdown': 0,
                   'reps': RepsSetting(1, 0, 0, 1),
                   'exercises': [ExerciseSetting('crosstrain', 30, 0, 0, 30)]},
}


plans = {
    'beginner': [['runeasy', ],
                 ['hillsprint', ],
                 ['intervals', ]],
    'intermediate': [['runeasy', ],
                     ['hillsprint', 'runeasy'],
                     ['intervals', ]],
    'advanced': [['runeasy', ],
                 ['hillsprint', 'intervals'],
                 ['intervals', ],
                 ['crosstrain', ]],
}

In [35]:
def rest_week(week, plan_length):
    '''
    int -> boolean

    Determine if current week is a rest week.

    Plans work on a 4 week block, with every 4th week being an easier week.
    Runner has at least 2 weeks, and a maximum of 5 weeks before they get an
    easier week.  So if they were on a 6 week plan they would only have an
    easier week on race week.

    Returns True if rest week and False if progression week.
    '''
    build_up = plan_length % 4
    if week <= build_up < 3:
        return False
    elif (week - build_up) % 4 == 0:
        return True
    else:
        return False

In [36]:
def create_plan(ability, length, days=[0, 1, 2]):
    """ Create bespoke training plan.
    int
    list
    --> dict
    plan = {'date': workout}
    """

    bespoke = {}

    for day, plan in zip(days, plans[ability]):
        for wk, progression in zip(range(length), cycle(plan)):
            ps = progress_settings[progression]
            workout = {}
            workout['warmup'] = Exercise('easy', ps['warmup'])
            workout['warmdown'] = Exercise('easy', ps['warmdown'])

            if rest_week(wk, length):
                multiple = wk - 1
            else:
                multiple = wk

#             if wk % ps['reps'].step_interval == 0:
#                 reps = ps['reps'].start + ps['reps'].step * multiple

            workout['reps'] = min(ps['reps'].start +
                                  ps['reps'].step * multiple, ps['reps'].max)

            workout['exercises'] = []
            for es in ps['exercises']:
                description = es.description
                time = min(es.start + es.step * multiple, es.max)
                workout['exercises'].append(Exercise(description, time))
            bespoke[f'wk{wk}-day{day}'] = workout

    return bespoke

In [44]:
plan = create_plan('advanced', 20,  [0, 2, 4, 5])

In [45]:
for session in plan:
    if 'day0' in session:
        pprint({session: {plan[session]['reps']: plan[session]['exercises']}})

{'wk0-day0': {1: [Exercise(name='easy', time=25)]}}
{'wk1-day0': {1: [Exercise(name='easy', time=30)]}}
{'wk2-day0': {1: [Exercise(name='easy', time=35)]}}
{'wk3-day0': {1: [Exercise(name='easy', time=40)]}}
{'wk4-day0': {1: [Exercise(name='easy', time=40)]}}
{'wk5-day0': {1: [Exercise(name='easy', time=50)]}}
{'wk6-day0': {1: [Exercise(name='easy', time=55)]}}
{'wk7-day0': {1: [Exercise(name='easy', time=60)]}}
{'wk8-day0': {1: [Exercise(name='easy', time=60)]}}
{'wk9-day0': {1: [Exercise(name='easy', time=70)]}}
{'wk10-day0': {1: [Exercise(name='easy', time=75)]}}
{'wk11-day0': {1: [Exercise(name='easy', time=80)]}}
{'wk12-day0': {1: [Exercise(name='easy', time=80)]}}
{'wk13-day0': {1: [Exercise(name='easy', time=90)]}}
{'wk14-day0': {1: [Exercise(name='easy', time=95)]}}
{'wk15-day0': {1: [Exercise(name='easy', time=100)]}}
{'wk16-day0': {1: [Exercise(name='easy', time=100)]}}
{'wk17-day0': {1: [Exercise(name='easy', time=100)]}}
{'wk18-day0': {1: [Exercise(name='easy', time=100)]}}

In [46]:
for session in plan:
    if 'day2' in session:
        pprint({session: {plan[session]['reps']: plan[session]['exercises']}})

{'wk0-day2': {6: [Exercise(name='hill', time=1.0)]}}
{'wk1-day2': {1: [Exercise(name='fast', time=0.75),
                  Exercise(name='easy', time=1)]}}
{'wk2-day2': {8: [Exercise(name='hill', time=1.5)]}}
{'wk3-day2': {1: [Exercise(name='fast', time=1.25),
                  Exercise(name='easy', time=1)]}}
{'wk4-day2': {8: [Exercise(name='hill', time=1.75)]}}
{'wk5-day2': {1: [Exercise(name='fast', time=1.75),
                  Exercise(name='easy', time=1)]}}
{'wk6-day2': {8: [Exercise(name='hill', time=2)]}}
{'wk7-day2': {1: [Exercise(name='fast', time=2),
                  Exercise(name='easy', time=1)]}}
{'wk8-day2': {8: [Exercise(name='hill', time=2)]}}
{'wk9-day2': {1: [Exercise(name='fast', time=2),
                  Exercise(name='easy', time=1)]}}
{'wk10-day2': {8: [Exercise(name='hill', time=2)]}}
{'wk11-day2': {1: [Exercise(name='fast', time=2),
                   Exercise(name='easy', time=1)]}}
{'wk12-day2': {8: [Exercise(name='hill', time=2)]}}
{'wk13-day2': {1: [Exer

In [47]:
for session in plan:
    if 'day4' in session:
        pprint({session: {plan[session]['reps']: plan[session]['exercises']}})

{'wk0-day4': {1: [Exercise(name='fast', time=0.5),
                  Exercise(name='easy', time=1)]}}
{'wk1-day4': {1: [Exercise(name='fast', time=0.75),
                  Exercise(name='easy', time=1)]}}
{'wk2-day4': {1: [Exercise(name='fast', time=1.0),
                  Exercise(name='easy', time=1)]}}
{'wk3-day4': {1: [Exercise(name='fast', time=1.25),
                  Exercise(name='easy', time=1)]}}
{'wk4-day4': {1: [Exercise(name='fast', time=1.25),
                  Exercise(name='easy', time=1)]}}
{'wk5-day4': {1: [Exercise(name='fast', time=1.75),
                  Exercise(name='easy', time=1)]}}
{'wk6-day4': {1: [Exercise(name='fast', time=2.0),
                  Exercise(name='easy', time=1)]}}
{'wk7-day4': {1: [Exercise(name='fast', time=2),
                  Exercise(name='easy', time=1)]}}
{'wk8-day4': {1: [Exercise(name='fast', time=2),
                  Exercise(name='easy', time=1)]}}
{'wk9-day4': {1: [Exercise(name='fast', time=2),
                  Exercise(name='

In [48]:
for session in plan:
    if 'day5' in session:
        pprint({session: {plan[session]['reps']: plan[session]['exercises']}})

{'wk0-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk1-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk2-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk3-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk4-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk5-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk6-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk7-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk8-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk9-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk10-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk11-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk12-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk13-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk14-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk15-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk16-day5': {1: [Exercise(name='crosstrain', time=30)]}}
{'wk17-